# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import numpy as np
from datetime import datetime
def date_sorter():
    
    cond11 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    cond12 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    cond1 = pd.concat([cond11,cond12])
    cond1.reset_index(inplace=True)
    cond1_id = cond1['level_0']
    #return cond1
    
    cond2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    cond2.reset_index(inplace=True)
    cond2_id = cond2['level_0']
    #return cond2
    
    cond3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    cond3.reset_index(inplace=True)
    cond3_id = cond3['level_0']
    #return cond3

    cond6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    cond6.reset_index(inplace=True)
    cond6_id = cond6['level_0']
    save=[]
    for i in cond6_id:
        if not(i in cond1_id.values):
            save.append(i)
    save = np.asarray(save)
    cond6 = cond6[cond6['level_0'].isin(save)]
    cond7_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    cond7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    cond7 = pd.concat([cond7_1,cond7_2])
    cond7.reset_index(inplace=True)
    cond7_id = cond7['level_0']
    existed=[]
    for i in cond7_id:
        if not((i in cond2_id.values) | (i in cond3_id.values) | (i in cond6_id.values)):
            existed.append(i)
    existed = np.asarray(existed)
    cond7 = cond7[cond7['level_0'].isin(existed)]
    
    s = cond1.level_0.values.tolist()+cond2.level_0.values.tolist()+cond3.level_0.values.tolist()+cond6.level_0.values.tolist()+cond7.level_0.values.tolist()
    s = np.asarray(s)
    
    cond1.columns=['level_0','match','month','day','year']
    cond1['year']=cond1['year'].apply(str)
    cond1['year']=cond1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    cond2[1] = cond2[1].apply(lambda x: x.replace(',',''))
    cond2['day'] = cond2[1].apply(lambda x:x.split(' ')[0])
    cond2['year'] = cond2[1].apply(lambda x:x.split(' ')[1])
    cond2.columns=['level_0','match','month','day-year','day','year']
    cond2.drop('day-year',axis=1,inplace=True) 
    
    cond3.columns=['level_0','match','day','month','year']
    cond3['day'] = cond3['day'].replace(np.nan,-99)
    cond3['day'] = cond3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    cond3['month'] = cond3.month.apply(lambda x: x[:3])
    cond3['month'] = pd.to_datetime(cond3.month, format='%b').dt.month
    
    cond6.columns=['level_0','match','month','year']
    cond6['day']=1
  
    cond7.columns=['level_0','match','year']
    cond7['day']=1
    cond7['month']=1
   
    final = pd.concat([cond1,cond2,cond3,cond6,cond7])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')
    
    myList = final['date']
    
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500))

    
    return answer
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64